In [5]:
import torch
from models import modules, net, senet
import rasterio
import numpy as np
import math
from rasterstats import zonal_stats

In [6]:
modelfile="pretrain/Block0_skip_model_110.pth.tar"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def loadmodel(modelfile, device):
    
    original_model = senet.senet154(pretrained=None)
    Encoder = modules.E_senet(original_model)
    model = net.model(Encoder, num_features=2048, block_channel = [256, 512, 1024, 2048])

    model.to(device)
    state_dict = torch.load(modelfile)['state_dict']

    del state_dict["E.Harm.dct"]
    del state_dict["E.Harm.weight"]
    del state_dict["E.Harm.bias"]

    model.load_state_dict(state_dict)

    return model

model=loadmodel(modelfile, device)
model.eval()

model(
  (E): E_senet(
    (base): Sequential(
      (0): Sequential(
        (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu3): ReLU(inplace=True)
        (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (1): Sequential(
        (0): SEBottleneck(
          (conv1): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): Bat

In [7]:
__imagenet_stats = {'mean': [0.485, 0.456, 0.406],
                    'std': [0.229, 0.224, 0.225]}

In [8]:
def normalize(images, means, stds):
    for t, m, s in zip(images, means, stds):
        t.sub_(m).div_(s)
    
    return images.float()

In [9]:
def one2two(i, xsize): #convert vector index to matrix index; x - column, y - row
  x= i % xsize
  y= i // xsize
  return (x, y)

In [10]:
def predict_batch(images_inbatch, model, device):
    batch=np.stack(images_inbatch, axis=0)
    
    batch=torch.from_numpy(batch)
    
    batch=normalize(batch, __imagenet_stats['mean'], __imagenet_stats['std'])
    
    batch=batch.to(device)
    with torch.no_grad():
        pred = model(batch)

    pred = torch.nn.functional.interpolate(pred,size=(440,440),mode='bilinear')
    pred = pred.detach().cpu().numpy()
    pred= pred*100
    
    pred=pred.squeeze()
    
    return pred

In [11]:
# predict by batchsize
def predict(inputfile, outputfile, model, input_size=440, padding=30, batchsize=2):
    with rasterio.open(inputfile) as src:
        w,h = src.width,src.height
        source=src.read().transpose(1,2,0)[:,:,0:3] #get first 3 bands
        profile = src.profile
    
    # predict
    crop_size=input_size-2*padding

    if h%crop_size==0:
        new_h=h
    else:
        new_h=h+crop_size-(h%crop_size)

    if w%crop_size==0:
        new_w=w
    else:
        new_w=w+crop_size-(w%crop_size)

    top_pad=padding
    bottom_pad=padding + new_h - h
    left_pad=padding
    right_pad=padding + new_w - w
    top_pad, left_pad, bottom_pad, right_pad

    # padding image
    source_pad=np.pad(source, ((top_pad, bottom_pad), (left_pad, right_pad), (0, 0)), mode='constant', constant_values=0)
    ylist=range(padding, source_pad.shape[0]-2*padding,crop_size)
    xlist=range(padding, source_pad.shape[1]-2*padding,crop_size)

    print('Predicting {} rows of {} columns...'.format(len(ylist), len(xlist)))
    
    outshape_pad=source_pad.shape[:-1]
    farm_pad=np.zeros(outshape_pad, dtype=np.uint8)

    size=len(ylist) * len(xlist)
    index=0
    while index<size:
        print('Batch number: {}/{}'.format(index // batchsize, math.ceil(size / batchsize)))
        arr=[]
        for i in range(batchsize):
            if (index+i<size):
                x, y = one2two(index+i, len(xlist))
                arr.append((ylist[y], xlist[x]))
        index+=batchsize

        images_inbatch=[]
        for (y,x) in arr:
            img=source_pad[y-padding:y+crop_size+padding,x-padding:x+crop_size+padding,:]
            img = np.array(img/255.0, dtype=np.float32).transpose(2,0,1)
            images_inbatch.append(img)

        pred=predict_batch(images_inbatch, model, device)

        for i, (y, x) in enumerate(arr):
            if len(pred.shape)==3: # batch, h, w
                farm_pad[y:y+crop_size,x:x+crop_size]=pred[i][padding:padding+crop_size, padding: padding+crop_size]
            else:
                farm_pad[y:y+crop_size,x:x+crop_size]=pred[padding:padding+crop_size, padding: padding+crop_size]

    # clean
    source_pad=None
    source=None

    # clip to the original shape        
    farm_final=farm_pad[padding:h+padding, padding:w+padding]
    
    # Write out
    profile.update(
        dtype=rasterio.float32,
        count=1,
        compress='lzw')

    #farm boundary
    with rasterio.open(outputfile, 'w', **profile) as dst:
        dst.write(farm_final, 1)


In [12]:
# inputfile='./test/Penang_airborne_10cm_utm47n-001.tif'
# outputfile='./test/Penang_airborne_10cm_utm47n-001_height_new.tif'

inputfile='./test/RGB_54366543.tif'
outputfile='./test/RGB_54366543_height.tif'

predict(inputfile, outputfile, model)

Predicting 31 rows of 37 columns...
Batch number: 0/574
Batch number: 1/574
Batch number: 2/574
Batch number: 3/574
Batch number: 4/574
Batch number: 5/574
Batch number: 6/574
Batch number: 7/574
Batch number: 8/574
Batch number: 9/574
Batch number: 10/574
Batch number: 11/574
Batch number: 12/574
Batch number: 13/574
Batch number: 14/574
Batch number: 15/574
Batch number: 16/574
Batch number: 17/574
Batch number: 18/574
Batch number: 19/574
Batch number: 20/574
Batch number: 21/574
Batch number: 22/574
Batch number: 23/574
Batch number: 24/574
Batch number: 25/574
Batch number: 26/574
Batch number: 27/574
Batch number: 28/574
Batch number: 29/574
Batch number: 30/574
Batch number: 31/574
Batch number: 32/574
Batch number: 33/574
Batch number: 34/574
Batch number: 35/574
Batch number: 36/574
Batch number: 37/574
Batch number: 38/574
Batch number: 39/574
Batch number: 40/574
Batch number: 41/574
Batch number: 42/574
Batch number: 43/574
Batch number: 44/574
Batch number: 45/574
Batch nu

In [15]:
# buildingfootprint='./test/all.shp'
# output_bf='./test/all_height_new.shp'

buildingfootprint='./test/RGB_54366543.shp'
output_bf='./test/RGB_54366543_height.shp'

import fiona
with fiona.open(buildingfootprint) as src:
    zs = zonal_stats(src, outputfile, stats='median', nodata=-32768)
    size=len(zs)
    schema=src.schema
    schema['properties']['height'] = 'float'
    
    with fiona.open(output_bf, 'w', crs=src.crs, driver=src.driver, schema=schema) as dst:
        for idx, f in enumerate(src):
            f['properties'].update(height=zs[idx]['median'])
            dst.write(f)
            print(f'Feature {idx}/{size}')

/home/skm/anaconda3/envs/im2ele/lib/python3.8/site-packages/rasterstats/main.py:156: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if 'Point' in geom.type:


Feature 0/6259
Feature 1/6259
Feature 2/6259
Feature 3/6259
Feature 4/6259
Feature 5/6259
Feature 6/6259
Feature 7/6259
Feature 8/6259
Feature 9/6259
Feature 10/6259
Feature 11/6259
Feature 12/6259
Feature 13/6259
Feature 14/6259
Feature 15/6259
Feature 16/6259
Feature 17/6259
Feature 18/6259
Feature 19/6259
Feature 20/6259
Feature 21/6259
Feature 22/6259
Feature 23/6259
Feature 24/6259
Feature 25/6259
Feature 26/6259
Feature 27/6259
Feature 28/6259
Feature 29/6259
Feature 30/6259
Feature 31/6259
Feature 32/6259
Feature 33/6259
Feature 34/6259
Feature 35/6259
Feature 36/6259
Feature 37/6259
Feature 38/6259
Feature 39/6259
Feature 40/6259
Feature 41/6259
Feature 42/6259
Feature 43/6259
Feature 44/6259
Feature 45/6259
Feature 46/6259
Feature 47/6259
Feature 48/6259
Feature 49/6259
Feature 50/6259
Feature 51/6259
Feature 52/6259
Feature 53/6259
Feature 54/6259
Feature 55/6259
Feature 56/6259
Feature 57/6259
Feature 58/6259
Feature 59/6259
Feature 60/6259
Feature 61/6259
Feature 62/6259
Fe